#### Create ABS Error and ABS usage to have to have mape calculation

In [164]:
import pandas as pd
import numpy as np
# show all columns
pd.set_option('display.max_columns', None)
# show all rows
pd.set_option('display.max_rows', None)

# file = './athena_sql_results/174global-settlement_mape_w_weather.csv'
file = './athena_sql_results/174global_ops.csv'
raw = pd.read_csv(file)
raw.head()

,proxy_date,zone,profile,customer_type,hour,Backcast,Forecast,Settlement
0,2024-06-06,NORTH,RESHIWR,NON_SOLAR,7,6.97,6.90,6.17
1,2024-06-18,NORTH,RESHIWR,NON_SOLAR,20,21.22,20.06,20.25
2,2024-06-13,NORTH,RESHIWR,NON_SOLAR,11,11.55,11.67,11.73
3,2024-06-07,NORTH,RESHIWR,NON_SOLAR,7,7.89,7.23,6.94
4,2024-06-10,NORTH,RESLOWR,NON_SOLAR,13,10.43,9.62,10.02


In [165]:
# Define the column mappings
column_mappings = [
    ['forecast', 'backcast', 'settlement', 'forecast_abs_error', 'backcast_abs_error', 'settlement_abs','forecast_mape', 'backcast_mape'],
    ['forecast_gross', 'backcast_gross', 'usage_final_gross', 'forecast_gross_abs_error','backcast_gross_abs_error', 'usage_final_gross_abs', 'forecast_gross_mape', 'backcast_gross_mape'],
    ['forecast_net', 'backcast_net', 'usage_final_net', 'forecast_net_abs_error', 'backcast_net_abs_error', 'usage_final_net_abs', 'forecast_net_mape', 'backcast_net_mape']
]

# Lowercase the DataFrame columns
raw.columns = raw.columns.str.lower()

cols = [
    col for mapping in column_mappings
    for col in mapping
    if col in raw.columns
]
cols
#if list cols empty then return error
if not cols:
    raise ValueError('No columns found in the dataframe, make sure to define mappings correctly')
sum_cols = {col: 'sum' for col in cols}
sum_cols

{'forecast': 'sum', 'backcast': 'sum', 'settlement': 'sum'}

In [84]:
# create an if statement that will check if specific columns are in the dataframe
# then store those specific columns in a list to reference when grouping the data
# list_of_cols = ['forecast', 'backcast','settlement', 'forecast_gross', 'backcast_gross', 'forecast_net', 'backcast_net', 'usage_final_gross', 'usage_final_net']
# cols = [col for col in list_of_cols if col in raw.columns]
# cols

#### Calculating ABS Error and ABS Usage Portfolio

In [166]:
# Aggregate by hour using proxy_date and hour then only use columns:
# forecast_gross, forecast_net, backcast_gross, backcast_net, usage_final_gross, usage_final_net, ams_actual_net, ams_actual_consumption
df_p = raw.copy()
df_p['proxy_date'] = pd.to_datetime(df_p['proxy_date']).dt.date
df_p['hour'] = df_p['hour'].astype(int)
df_p = df_p.groupby(['proxy_date', 'hour']).agg(sum_cols).reset_index()

# now from the cols list create a new list that will be used to create the Mforecas ABS error and backcast ABS error
# this will be done for both gross and net
# this will be done by subtracting the forecast from the usage_final and backcast from the usage_final
# then taking the absolute value of the result

# Perform the calculations based on the columns present in the DataFrame
for mapping in column_mappings:
    f, b, s, f_abs_e, b_abs_e, s_abs_e, f_m, b_m = mapping
    if all(col in df_p.columns for col in [f, b, s]):
        df_p[f_abs_e] = abs(df_p[f] - df_p[s])
        df_p[b_abs_e] = abs(df_p[b] - df_p[s])
        df_p[s_abs_e] = abs(df_p[s])


# if 'forecast_gross' in cols and 'usage_final_gross' in cols:
#     df['forecast_gross_abs_error'] = abs(df['forecast_gross'] - df['usage_final_gross'])
#     df['usage_final_gross_abs'] = abs(df['usage_final_gross'])
# if 'forecast_net' in cols and 'usage_final_net' in cols:
#     df['forecast_net_abs_error'] = abs(df['forecast_net'] - df['usage_final_net'])
#     df['usage_final_net_abs'] = abs(df['usage_final_net'])
# if 'backcast_gross' in cols and 'usage_final_gross' in cols:
#     df['backcast_gross_abs_error'] = abs(df['backcast_gross'] - df['usage_final_gross'])
# if 'backcast_net' in cols and 'usage_final_net' in cols:
#     df['backcast_net_abs_error'] = abs(df['backcast_net'] - df['usage_final_net'])
# if 'forecast' in cols and 'settlement' in cols:
#     df['forecast_abs_error'] = abs(df['forecast'] - df['settlement'])
#     df['settlement_abs'] = abs(df['settlement'])
# if 'backcast' in cols and 'settlement' in cols:
#     df['backcast_abs_error'] = abs(df['backcast'] - df['settlement'])


# Add columns for forecast ABS error and backcast ABS error for both gross and net
# df['forecast_gross_abs_error'] = abs(df['forecast_gross'] - df['usage_final_gross'])
# df['forecast_net_abs_error'] = abs(df['forecast_net'] - df['usage_final_net'])
# df['backcast_gross_abs_error'] = abs(df['backcast_gross'] - df['usage_final_gross'])
# df['backcast_net_abs_error'] = abs(df['backcast_net'] - df['usage_final_net'])
# df['usage_final_gross_abs'] = abs(df['usage_final_gross'])
# df['usage_final_net_abs'] = abs(df['usage_final_net'])
df_p.head()


,proxy_date,hour,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
0,2024-01-01,1,0.0,0.0,34.99,34.99,34.99,34.99
1,2024-01-01,2,0.0,0.0,35.22,35.22,35.22,35.22
2,2024-01-01,3,0.0,0.0,35.12,35.12,35.12,35.12
3,2024-01-01,4,0.0,0.0,35.02,35.02,35.02,35.02
4,2024-01-01,5,0.0,0.0,36.45,36.45,36.45,36.45


In [167]:
# order by proxy_date and hour (0-23) so that hour goes as follow per day 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23
df_p = df_p.sort_values(['proxy_date', 'hour'])
df_p.head()

,proxy_date,hour,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
0,2024-01-01,1,0.0,0.0,34.99,34.99,34.99,34.99
1,2024-01-01,2,0.0,0.0,35.22,35.22,35.22,35.22
2,2024-01-01,3,0.0,0.0,35.12,35.12,35.12,35.12
3,2024-01-01,4,0.0,0.0,35.02,35.02,35.02,35.02
4,2024-01-01,5,0.0,0.0,36.45,36.45,36.45,36.45


In [168]:
# cols = []
# for mapping in column_mappings:
#     f, b, s, f_abs_e, b_abs_e, s_abs_e, f_m, b_m = mapping
#     if f in df.columns and b in df.columns and s in df.columns:
#         cols.append(f), cols.append(b), cols.append(s), cols.append(f_abs_e), cols.append(b_abs_e), cols.append(s_abs_e)  
# cols


In [169]:
cols = [
    col for mapping in column_mappings
    for col in mapping
    if col in df_p.columns
]
cols

['forecast',
 'backcast',
 'settlement',
 'forecast_abs_error',
 'backcast_abs_error',
 'settlement_abs']

In [170]:
sum_cols = {col: 'sum' for col in cols}
sum_cols

{'forecast': 'sum',
 'backcast': 'sum',
 'settlement': 'sum',
 'forecast_abs_error': 'sum',
 'backcast_abs_error': 'sum',
 'settlement_abs': 'sum'}

#### Calculate daily MAPE

In [183]:
# aggregate to day to get the daily mape for forecast and backcast for both gross and net
# df['proxy_date'] = pd.to_datetime(df['proxy_date'])
d_mape = df_p.copy()
# d_mape['day'] = d_mape['proxy_date'].dt.date
d_mape = d_mape.groupby('proxy_date').agg(sum_cols).reset_index()

for mapping in column_mappings:
    f, b, s, f_abs_e, b_abs_e, s_abs_e, f_m, b_m = mapping
    if all(col in d_mape.columns for col in [s_abs_e, f_abs_e, b_abs_e]):
        d_mape[f_m] = d_mape[f_abs_e] / d_mape[s_abs_e]
        d_mape[b_m] = d_mape[b_abs_e] / d_mape[s_abs_e]
d_mape.head()
# d_mape['forecast_gross_mape'] = d_mape['forecast_gross_abs_error'] / d_mape['usage_final_gross_abs']
# d_mape['forecast_net_mape'] = d_mape['forecast_net_abs_error'] / d_mape['usage_final_net_abs']
# d_mape['backcast_gross_mape'] = d_mape['backcast_gross_abs_error'] / d_mape['usage_final_gross_abs']
# d_mape['backcast_net_mape'] = d_mape['backcast_net_abs_error'] / d_mape['usage_final_net_abs']

,proxy_date,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs,forecast_mape,backcast_mape
0,2024-01-01,0.0,0.0,814.03,814.03,814.03,814.03,1.0,1.0
1,2024-01-02,0.0,0.0,1061.63,1061.63,1061.63,1061.63,1.0,1.0
2,2024-01-03,0.0,0.0,797.62,799.62,799.62,799.62,1.0,1.0
3,2024-01-04,0.0,0.0,777.93,790.87,790.87,790.87,1.0,1.0
4,2024-01-05,0.0,0.0,747.61,747.61,747.61,747.61,1.0,1.0


#### Drop NA or columns that do not make up mape 

In [190]:
# remove any Nan or inf  rows based on the mape columns
d_mape = d_mape.replace([np.inf, -np.inf], np.nan)
d_mape = d_mape.dropna()

# Remove rows where both forecast and backcast columns contain 0.0
forecast_cols = [col for col in cols if 'forecast' in col]
backcast_cols = [col for col in cols if 'backcast' in col]

for f_col, b_col in zip(forecast_cols, backcast_cols):
    if f_col in d_mape.columns and b_col in d_mape.columns:
        d_mape = d_mape[~((d_mape[f_col] == 0.0) & (d_mape[b_col] == 0.0))]
d_mape 

,proxy_date,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs,forecast_mape,backcast_mape
149,2024-05-29,998.39,961.22,1009.29,199.34,82.81,1011.01,0.197169,0.081908
150,2024-05-30,1042.41,1065.45,1214.77,174.06,149.32,1214.77,0.143286,0.122920
151,2024-05-31,1001.30,868.57,936.22,128.92,141.67,937.90,0.137456,0.151050
152,2024-06-01,1087.03,1022.17,1095.06,76.03,120.41,1095.06,0.069430,0.109957
153,2024-06-02,1247.30,1202.06,1307.17,100.27,128.65,1307.17,0.076708,0.098419
154,2024-06-03,1284.28,1253.10,1355.00,105.38,102.62,1355.00,0.077771,0.075734
155,2024-06-04,1340.21,1435.88,1467.73,135.78,75.83,1467.73,0.092510,0.051665
156,2024-06-05,1369.94,1261.10,1260.34,193.16,64.36,1260.34,0.153260,0.051066
157,2024-06-06,1294.20,1292.04,1280.07,87.87,24.15,1280.07,0.068645,0.018866
158,2024-06-07,1270.48,1431.99,1457.36,186.88,44.47,1457.36,0.128232,0.030514


#### Dynamic column ordering

In [191]:
# Extract columns that contain 'mape'
mape_columns = [col for col in d_mape.columns if 'mape' in col]
# Extract other columns
other_columns = [col for col in d_mape.columns if 'mape' not in col]
# Define the desired order for 'mape' columns
mape_insert_position = 1
# Create the new column order
new_columns_order = other_columns[:mape_insert_position] + mape_columns + other_columns[mape_insert_position:]
# Reorder the DataFrame columns
d_mape = d_mape[new_columns_order]
d_mape.head()


,proxy_date,forecast_mape,backcast_mape,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
149,2024-05-29,0.197169,0.081908,998.39,961.22,1009.29,199.34,82.81,1011.01
150,2024-05-30,0.143286,0.122920,1042.41,1065.45,1214.77,174.06,149.32,1214.77
151,2024-05-31,0.137456,0.151050,1001.30,868.57,936.22,128.92,141.67,937.90
152,2024-06-01,0.069430,0.109957,1087.03,1022.17,1095.06,76.03,120.41,1095.06
153,2024-06-02,0.076708,0.098419,1247.30,1202.06,1307.17,100.27,128.65,1307.17


In [194]:
# d_mape = d_mape[['proxy_date', 'forecast_gross_mape', 
#         'backcast_gross_mape', 'forecast_net_mape', 'backcast_net_mape','forecast_gross', 
#         'forecast_gross_abs_error',
#        'forecast_net', 'forecast_net_abs_error', 'backcast_gross',
#        'backcast_gross_abs_error', 'backcast_net', 'backcast_net_abs_error',
#        'usage_final_gross', 'usage_final_gross_abs', 'usage_final_net',
#        'usage_final_net_abs']] 
# d_mape.head()
cols = [
    col for mapping in column_mappings
    for col in mapping
    if col in raw.columns
]
cols
#if list cols empty then return error
if not cols:
    raise ValueError('No columns found in the dataframe, make sure to define mappings correctly')
sum_cols = {col: 'sum' for col in cols}

#### Zone Calculation of ABS Error and ABS Usage

In [195]:
# raw dataframe aggregate by proxy_date, hour, and zone
df_zone = raw.copy()
df_zone['proxy_date'] = pd.to_datetime(df_zone['proxy_date']).dt.date
df_zone['hour'] = df_zone['hour'].astype(int)
df_zone = df_zone.groupby(['proxy_date', 'hour', 'zone']).agg(sum_cols).reset_index()

for mapping in column_mappings:
    f, b, s, f_abs_e, b_abs_e, s_abs_e, f_m, b_m = mapping
    if all(col in df_zone.columns for col in [f, b, s]):
        df_zone[f_abs_e] = abs(df_zone[f] - df_zone[s])
        df_zone[b_abs_e] = abs(df_zone[b] - df_zone[s])
        df_zone[s_abs_e] = abs(df_zone[s])

# Add columns for forecast ABS error and backcast ABS error for both gross and net
# df_zone['forecast_gross_abs_error'] = abs(df_zone['forecast_gross'] - df_zone['usage_final_gross'])
# df_zone['forecast_net_abs_error'] = abs(df_zone['forecast_net'] - df_zone['usage_final_net'])
# df_zone['backcast_gross_abs_error'] = abs(df_zone['backcast_gross'] - df_zone['usage_final_gross'])
# df_zone['backcast_net_abs_error'] = abs(df_zone['backcast_net'] - df_zone['usage_final_net'])
# df_zone['usage_final_gross_abs'] = abs(df_zone['usage_final_gross'])
# df_zone['usage_final_net_abs'] = abs(df_zone['usage_final_net'])
df_zone.head()

,proxy_date,hour,zone,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
0,2024-01-01,1,HOUSTON,0.0,0.0,8.27,8.27,8.27,8.27
1,2024-01-01,1,NORTH,0.0,0.0,19.97,19.97,19.97,19.97
2,2024-01-01,1,SOUTH,0.0,0.0,3.93,3.93,3.93,3.93
3,2024-01-01,1,WEST,0.0,0.0,2.82,2.82,2.82,2.82
4,2024-01-01,2,HOUSTON,0.0,0.0,7.72,7.72,7.72,7.72


In [196]:
df_zone = df_zone.sort_values(['proxy_date', 'hour'])
df_zone.head()

,proxy_date,hour,zone,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
0,2024-01-01,1,HOUSTON,0.0,0.0,8.27,8.27,8.27,8.27
1,2024-01-01,1,NORTH,0.0,0.0,19.97,19.97,19.97,19.97
2,2024-01-01,1,SOUTH,0.0,0.0,3.93,3.93,3.93,3.93
3,2024-01-01,1,WEST,0.0,0.0,2.82,2.82,2.82,2.82
4,2024-01-01,2,HOUSTON,0.0,0.0,7.72,7.72,7.72,7.72


In [197]:
cols = [
    col for mapping in column_mappings
    for col in mapping
    if col in df_zone.columns
]
sum_cols = {col: 'sum' for col in cols}


#### Calculate MAPE for each zone

In [198]:
# get the daily mape by zone
d_mape_zone = df_zone.copy()
# d_mape_zone['day'] = d_mape_zone['proxy_date'].dt.date
d_mape_zone = d_mape_zone.groupby(['proxy_date', 'zone']).agg(sum_cols).reset_index()

for mapping in column_mappings:
    f, b, s, f_abs_e, b_abs_e, s_abs_e, f_m, b_m = mapping
    if all(col in d_mape.columns for col in [s_abs_e, f_abs_e, b_abs_e]):
        d_mape_zone[f_m] = d_mape_zone[f_abs_e] / d_mape_zone[s_abs_e]
        d_mape_zone[b_m] = d_mape_zone[b_abs_e] / d_mape_zone[s_abs_e]

# d_mape_zone['forecast_gross_mape'] = d_mape_zone['forecast_gross_abs_error'] / d_mape_zone['usage_final_gross_abs']
# d_mape_zone['forecast_net_mape'] = d_mape_zone['forecast_net_abs_error'] / d_mape_zone['usage_final_net_abs']
# d_mape_zone['backcast_gross_mape'] = d_mape_zone['backcast_gross_abs_error'] / d_mape_zone['usage_final_gross_abs']
# d_mape_zone['backcast_net_mape'] = d_mape_zone['backcast_net_abs_error'] / d_mape_zone['usage_final_net_abs']
d_mape_zone.head()

,proxy_date,zone,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs,forecast_mape,backcast_mape
0,2024-01-01,HOUSTON,0.0,0.0,175.01,175.01,175.01,175.01,1.0,1.0
1,2024-01-01,NORTH,0.0,0.0,518.55,518.55,518.55,518.55,1.0,1.0
2,2024-01-01,SOUTH,0.0,0.0,65.99,83.09,83.09,83.09,1.0,1.0
3,2024-01-01,WEST,0.0,0.0,54.48,61.62,61.62,61.62,1.0,1.0
4,2024-01-02,HOUSTON,0.0,0.0,227.52,227.52,227.52,227.52,1.0,1.0


In [199]:
# remove any Nan or inf  rows based on the mape columns
d_mape_zone = d_mape_zone.replace([np.inf, -np.inf], np.nan)
d_mape_zone = d_mape_zone.dropna()


# Remove rows where both forecast and backcast columns contain 0.0
forecast_cols = [col for col in cols if 'forecast' in col]
backcast_cols = [col for col in cols if 'backcast' in col]

for f_col, b_col in zip(forecast_cols, backcast_cols):
    if f_col in d_mape_zone.columns and b_col in d_mape_zone.columns:
        d_mape_zone = d_mape_zone[~((d_mape_zone[f_col] == 0.0) & (d_mape_zone[b_col] == 0.0))]
d_mape_zone 

d_mape_zone.head()

,proxy_date,zone,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs,forecast_mape,backcast_mape
596,2024-05-29,HOUSTON,406.13,373.07,392.52,78.99,39.19,392.52,0.201238,0.099842
597,2024-05-29,NORTH,442.92,448.31,472.12,81.30,34.19,472.12,0.172202,0.072418
598,2024-05-29,SOUTH,115.34,104.34,102.91,30.95,14.61,136.05,0.227490,0.107387
599,2024-05-29,WEST,34.00,35.50,41.74,19.70,13.34,66.90,0.294469,0.199402
600,2024-05-30,HOUSTON,414.86,427.60,491.90,78.10,64.30,491.90,0.158772,0.130718


#### Dynamic column ordering

In [201]:
# Extract columns that contain 'mape'
mape_columns = [col for col in d_mape_zone.columns if 'mape' in col]
# Extract other columns
other_columns = [col for col in d_mape_zone.columns if 'mape' not in col]
# Define the desired order for 'mape' columns
mape_insert_position = 2
# Create the new column order
new_columns_order = other_columns[:mape_insert_position] + mape_columns + other_columns[mape_insert_position:]
# Reorder the DataFrame columns
d_mape_zone = d_mape_zone[new_columns_order]
d_mape_zone.head()

,proxy_date,zone,forecast_mape,backcast_mape,forecast,backcast,settlement,forecast_abs_error,backcast_abs_error,settlement_abs
596,2024-05-29,HOUSTON,0.201238,0.099842,406.13,373.07,392.52,78.99,39.19,392.52
597,2024-05-29,NORTH,0.172202,0.072418,442.92,448.31,472.12,81.30,34.19,472.12
598,2024-05-29,SOUTH,0.227490,0.107387,115.34,104.34,102.91,30.95,14.61,136.05
599,2024-05-29,WEST,0.294469,0.199402,34.00,35.50,41.74,19.70,13.34,66.90
600,2024-05-30,HOUSTON,0.158772,0.130718,414.86,427.60,491.90,78.10,64.30,491.90


In [202]:
cols = [
    col for col in d_mape_zone.columns
    for mapping in column_mappings
    if col in mapping
]
cols

['forecast_mape',
 'backcast_mape',
 'forecast',
 'backcast',
 'settlement',
 'forecast_abs_error',
 'backcast_abs_error',
 'settlement_abs']

#### Zonal Pivot

In [203]:
# pivot the d_mape_zone dataframe to have the zone as columns
d_mape_zone_pivot = d_mape_zone.pivot(index='proxy_date', columns='zone', values=cols)
# Flatten the MultiIndex columns
d_mape_zone_pivot.columns = ['_'.join(map(str, col)).strip() for col in d_mape_zone_pivot.columns.values]
# Reset the index if needed
d_mape_zone_pivot.reset_index(inplace=True)
d_mape_zone_pivot.head()

,proxy_date,forecast_mape_HOUSTON,forecast_mape_NORTH,forecast_mape_SOUTH,forecast_mape_WEST,backcast_mape_HOUSTON,backcast_mape_NORTH,backcast_mape_SOUTH,backcast_mape_WEST,forecast_HOUSTON,forecast_NORTH,forecast_SOUTH,forecast_WEST,backcast_HOUSTON,backcast_NORTH,backcast_SOUTH,backcast_WEST,settlement_HOUSTON,settlement_NORTH,settlement_SOUTH,settlement_WEST,forecast_abs_error_HOUSTON,forecast_abs_error_NORTH,forecast_abs_error_SOUTH,forecast_abs_error_WEST,backcast_abs_error_HOUSTON,backcast_abs_error_NORTH,backcast_abs_error_SOUTH,backcast_abs_error_WEST,settlement_abs_HOUSTON,settlement_abs_NORTH,settlement_abs_SOUTH,settlement_abs_WEST
0,2024-05-29,0.201238,0.172202,0.227490,0.294469,0.099842,0.072418,0.107387,0.199402,406.13,442.92,115.34,34.00,373.07,448.31,104.34,35.50,392.52,472.12,102.91,41.74,78.99,81.30,30.95,19.70,39.19,34.19,14.61,13.34,392.52,472.12,136.05,66.90
1,2024-05-30,0.158772,0.234384,0.148384,0.246284,0.130718,0.114588,0.098947,0.285642,414.86,466.63,118.58,42.34,427.60,472.58,124.80,40.47,491.90,531.82,134.95,56.10,78.10,124.65,20.71,14.58,64.30,60.94,13.81,16.91,491.90,531.82,139.57,59.20
2,2024-05-31,0.174247,0.207502,0.149513,0.252378,0.179356,0.124658,0.146966,0.282657,411.69,441.77,114.10,33.74,327.01,397.85,103.96,39.75,368.04,402.06,122.84,43.28,64.13,84.91,19.96,15.92,66.01,51.01,19.62,17.83,368.04,409.20,133.50,63.08
3,2024-06-01,0.143713,0.192575,0.129936,0.310482,0.095268,0.120029,0.116004,0.300842,417.78,500.73,123.54,44.98,412.38,451.67,116.39,41.73,437.19,480.80,131.96,45.11,62.83,92.59,17.44,20.29,41.65,57.71,15.57,19.66,437.19,480.80,134.22,65.35
4,2024-06-02,0.149949,0.156014,0.128364,0.272427,0.071158,0.135582,0.104744,0.245543,458.67,604.08,134.00,50.55,497.94,514.21,137.49,52.42,533.18,553.54,150.95,69.50,79.95,86.36,19.51,18.95,37.94,75.05,15.92,17.08,533.18,553.54,151.99,69.56


In [149]:
# client_name = '174global'
# if os.path.exists(file_path):
#     with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
#         raw.to_excel(writer, sheet_name=f'{client_name}_raw', index=False)
#         df.to_excel(writer, sheet_name='hourly_portfolio', index=False)
#         d_mape.to_excel(writer, sheet_name='daily_portfolio_mape', index=False)
#         df_zone.to_excel(writer, sheet_name='hourly_zonal', index=False)
#         d_mape_zone.to_excel(writer, sheet_name='daily_zonal_mape_zone', index=False)
# else:
#     with pd.ExcelWriter(file_path, mode='w') as writer:
#         raw.to_excel(writer, sheet_name=f'{client_name}_raw', index=False)
#         df.to_excel(writer, sheet_name='hourly_portfolio', index=False)
#         d_mape.to_excel(writer, sheet_name='daily_portfolio_mape', index=False)
#         df_zone.to_excel(writer, sheet_name='hourly_zonal', index=False)
#         d_mape_zone.to_excel(writer, sheet_name='daily_zonal_mape_zone', index=False)

#### Save all dataframes to ExcelSpreadsheet

In [204]:
client_name = '174global'
# save multiple dataframes to the same excel sheet, if the sheet does not exist it will be created
# create empty excel sheet


with pd.ExcelWriter(f'./output/{client_name}_ops_performance.xlsx', mode='w') as writer:  
    raw.to_excel(writer, sheet_name=f'{client_name}_raw', index=False)
    df_p.to_excel(writer, sheet_name=f'hourly_portfolio', index=False)
    d_mape.to_excel(writer, sheet_name=f'daily_portfolio_mape', index=False)
    df_zone.to_excel(writer, sheet_name=f'hourly_zonal', index=False)
    d_mape_zone_pivot.to_excel(writer, sheet_name=f'daily_zonal_mape_zone', index=False)

In [208]:
raw = ['test']
df_p = {'test2'}
name = 'jp_'
def save_to_excelspreadsheet(client_name: str, **kwargs):
    print(client_name)
    print(kwargs)
    for sheet_name, df in kwargs.items():
        print(sheet_name, df)

save_to_excelspreadsheet(
    client_name='174global',
    any_name=raw,
    df_p=df_p,
)

174global
{'any_name': ['test'], 'df_p': {'test2'}}
any_name ['test']
df_p {'test2'}
